Now we may consider fitting a random forest to our model. We will proceed with the same filters as done in our Logistic model (minimum minutes played and games played)

In [1]:
import pandas as pd
import numpy as np

In [2]:
nba_train = pd.read_csv('Data_Scripting_Cleaning/Full_data/Training_Sets/nba_train.csv')
nba_test = pd.read_csv('Data_Scripting_Cleaning/Full_data/Test_Sets/nba_test.csv')


In [3]:
min_minutes = nba_train[(nba_train['all_nba_c_year']==1)].MP.min()
min_G = nba_train[(nba_train['all_nba_c_year']==1)].G.min()
nba_filt_train = nba_train[(nba_train['MP']>=min_minutes) & (nba_train['G']>=min_G)]
nba_filt_test = nba_test[(nba_test['MP']>=min_minutes) & (nba_test['G']>=min_G)]

y_train = nba_filt_train['all_nba_c_year']

y_test = nba_filt_test['all_nba_c_year']

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Now we will fit a random forest model to the data. We will fit this data into a pipeline to scale the data and then fit the model.

num_features = ['Age','G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS',
       'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'W',
       'num_all_nba']
cat_features = ['Tm']


preprocessor = ColumnTransformer(
    [("select", "passthrough", num_features),
     ("ohe", OneHotEncoder(handle_unknown="ignore"), cat_features)],
     remainder="drop"
)

preprocessor.fit(nba_filt_train)

ColumnTransformer(transformers=[('select', 'passthrough',
                                 ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
                                  '3P', '3PA', '3P%', '2P', '2PA', '2P%',
                                  'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
                                  'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
                                  'PTS', 'PER', 'TS%', '3PAr', 'FTr', ...]),
                                ('ohe', OneHotEncoder(handle_unknown='ignore'),
                                 ['Tm'])])

In [5]:
X_train = pd.DataFrame(preprocessor.transform(nba_filt_train), columns = preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(nba_filt_test), columns = preprocessor.get_feature_names_out())

To fit this model we will consider a grid of hyper-parameters. Since this grid is quite large (7200), we will utilize a randomized search where we consider 500 random subsets of the hyper-parameters.

We can also consider what these hyper-parameters mean.
1) n_estimators: How many decision trees will be created in total
2) max_depth: The longest path between the root node and the leaf. This is essentially controlling how many splits are allowed.
3) min_samples_lead: Minimum number of samples to required for a leaf node. This determines how specific a leaf can be.
4) min_samples_split: Minimum number of samples a node must have for it to be split
5) max_features: Maximum number features randomly selected for each tree.

In [6]:
#Now we can fit our Random Forest model to the data.
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestClassifier(random_state=0)
param_grid = {
   'n_estimators': [100, 200, 300, 400, 500],
   'max_depth': [5, 10, 15, 20, 25],
   'min_samples_split': [2, 5, 10, 15, 20, 25, 30, 40],
   'min_samples_leaf': [1, 2, 5, 10, 15, 20],
   'max_features': [ 'sqrt', 'log2', 0.2, .4, .8, 1.0]
}


CV_rfc = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, cv= 5,
                            random_state=0, n_iter=1000)
CV_rfc.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, CV_rfc.predict(X_test), target_names=['Not All-NBA', 'All-NBA']))

In [ ]:
pd.crosstab(y_test, CV_rfc.predict(X_test), rownames=['Actual'], colnames=['Predicted'], margins=True)

In [ ]:
coef_df = pd.DataFrame({'coefs':CV_rfc.best_estimator_.feature_importances_,
                          'features':X_train.columns})
coef_df_nz = coef_df[coef_df['coefs']!=0]

In [ ]:
import altair as alt
alt.Chart(coef_df_nz).mark_bar().encode(
    y='coefs',
    x=alt.Y('features', sort='-y'))